In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# load preprocessed csv file 
if os.path.exists('data/preprocessed_df.csv'):
    df = pd.read_csv('data/preprocessed_df.csv')
    print('Preprocessed_df.csv successfully opened')
else:
    raise FileNotFoundError('File was not found!')

Preprocessed_df.csv successfully opened


In [3]:
y = df['Churn'] # traget output

In [4]:
y.value_counts() # count unique values in target features - checks for class imbalance 

Churn
0    4682
1     948
Name: count, dtype: int64

In [5]:
x = df.drop(columns=['Churn']).copy() #drop target column from dataset
x

,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
0,4.0,SmartPhone,3,6.0,Debit Card,Female,3.0,3,Laptop & Accessory,2,Single,9,1,11.0,1.0,1.0,5.0,160
1,NaN,Phone,1,8.0,UPI,Male,3.0,4,Mobile,3,Single,7,1,15.0,0.0,1.0,0.0,121
2,NaN,Phone,1,30.0,Debit Card,Male,2.0,4,Mobile,3,Single,6,1,14.0,0.0,1.0,3.0,120
3,0.0,Phone,3,15.0,Debit Card,Male,2.0,4,Laptop & Accessory,5,Single,8,0,23.0,0.0,1.0,3.0,134
4,0.0,Phone,1,12.0,CC,Male,NaN,3,Mobile,5,Single,3,0,11.0,1.0,1.0,3.0,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5625,10.0,Computer,1,30.0,Credit Card,Male,3.0,2,Laptop & Accessory,1,Married,6,0,18.0,1.0,2.0,4.0,151
5626,13.0,SmartPhone,1,13.0,Credit Card,Male,3.0,5,Fashion,5,Married,6,0,16.0,1.0,2.0,NaN,225
5627,1.0,SmartPhone,1,11.0,Debit Card,Male,3.0,2,Laptop & Accessory,4,Married,3,1,21.0,1.0,2.0,4.0,186
5628,23.0,Computer,3,9.0,Credit Card,Male,4.0,5,Laptop & Accessory,4,Married,4,0,15.0,2.0,2.0,9.0,179


In [6]:
# import libraries
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
from sklearn.preprocessing import RobustScaler,PolynomialFeatures,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.utils import compute_class_weight
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier

In [7]:
# splits the x and y datasets into training and testing sets
x_train,x_test,y_train,y_test = train_test_split(
    x,y,test_size=0.3,random_state=42,stratify=y
)
# save train and test splits 
x_train.to_parquet('data/x_train.parquet',index=False)
x_test.to_parquet('data/x_test.parquet',index=False)
y_train.to_frame('y_train').to_parquet('data/y_train.parquet',index=False)
y_test.to_frame('y_test').to_parquet('data/y_test.parquet',index=False)


In [8]:
# divide columns into numerical and categorical columns
numeric_cols = x_train.select_dtypes(include='number').columns 
categorical_cols = x_train.select_dtypes(include='object').columns

cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=42) # cross - validation

# preprocessing
num_pipe = Pipeline(steps=[
    ('impute',SimpleImputer(strategy='mean')),
    ('scaler',RobustScaler()),
    ('poly',PolynomialFeatures(include_bias=False))
])

cat_pipe = Pipeline(steps=[
    ('impute',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num',num_pipe,numeric_cols),
    ('cat',cat_pipe,categorical_cols)
],verbose_feature_names_out=False)


# sample weights to handle class imbalance
sample_weights = compute_class_weight(class_weight='balanced',y=y_train,classes=np.unique(y))


In [9]:

# models and hyperparameter tuning
models_grid = {
    'Logistic_Regression' : {
    'model': LogisticRegression(penalty='l2',solver='lbfgs',random_state=42,n_jobs=-1,class_weight='balanced'),
    'params' : {
        'preprocessor__num__poly__degree' : [1,2],
        'classifier__C' : [0.5,0.8,1.0,3.0],
        'classifier__max_iter' : [400,500,600]
    }},
    'Random_Forest' : {
        'model' : RandomForestClassifier(n_jobs=-1,random_state=42,class_weight='balanced'),
        'params' : {
            'preprocessor__num__poly__degree' : [1,2],
            'classifier__n_estimators' : [100,120,150],
            'classifier__max_depth' : [6,8,10],
            'classifier__min_samples_split' : [3,5,7]
        }
    },
    'Decision_Trees' : {
        'model' : DecisionTreeClassifier(random_state=42,class_weight='balanced'),
        'params' : {
            'preprocessor__num__poly__degree' : [1,2],
            'classifier__max_depth' : [7,9,11],
            'classifier__min_samples_split' : [2,5,8]
        }
    },
    'XGBClassifier' : {
        'model' : XGBClassifier(objective='binary:logistic',verbosity=0,scale_pos_weight=sample_weights[0]/sample_weights[1]),
        'params' : {
            'preprocessor__num__poly__degree' : [1,2],
            'classifier__n_estimators' : [100,150,200],
            'classifier__max_depth' : [7,9,11],
            'classifier__learning_rate' : [0.3,0.5,0.7],
            'classifier__reg_lambda' : [0.1,0.3,0.5],
        }
    }
}
best_score = -float('inf')
best_name = None
best_estimator = None

results = {}
for name,model in models_grid.items():
    pipe = Pipeline(steps=[
        ('preprocessor',preprocessor),
        ('classifier',model['model'])
    ])

# grid search
    model = GridSearchCV(estimator=pipe,
                    param_grid=model['params'],
                    cv = cv,
                    refit = True,
                    scoring= 'f1',
                    return_train_score=True,
                    n_jobs=-1,
                    verbose=2,
                    error_score='raise')
    
    # Training models
    print(f'Training model using {name}: (This may take a while)...')
    model.fit(x_train,y_train)

    results[name] = {
        'best_score' : model.best_score_,
        'best_params' : model.best_params_
    }
    if model.best_score_ > best_score:
        best_score = model.best_score_
        best_name = name
        best_estimator = model.best_estimator_

    # save models to joblib
    import joblib
    joblib.dump(model.best_estimator_,f'models/{name}_best_model.pkl')
    print('Best model estimator successfully saved')

    # save results to a json file
    import json
    with open('models/model_results.json','w') as file:
        json.dump(results,file,indent=4)

    print('-'*50)

Training model using Logistic_Regression: (This may take a while)...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best model estimator successfully saved
--------------------------------------------------
Training model using Random_Forest: (This may take a while)...
Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best model estimator successfully saved
--------------------------------------------------
Training model using Decision_Trees: (This may take a while)...
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best model estimator successfully saved
--------------------------------------------------
Training model using XGBClassifier: (This may take a while)...
Fitting 5 folds for each of 162 candidates, totalling 810 fits
Best model estimator successfully saved
--------------------------------------------------
